In [1]:
# Loading in functions
from non_max_surpression import NonMaxSurpression
from model import DogDetectorModel
from image_to_tensor import to_tensor
import pandas as pd

# Loading libraries
import torch 
import os

## Model Loading

In [2]:
# Initializing the model
model = DogDetectorModel()

In [3]:
# Initializing Non Max Surpression with default parameters
nms = NonMaxSurpression()

In [4]:
# Setting the device
DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

CPU_DEVICE = torch.device("cpu")

In [5]:
# Loading the checkpoint
checkpoint = torch.load('dog-detection-model.pt')

# Loading the model weights
model.load_state_dict(checkpoint['model_state_dict'])

# Setting model to evaluation model
model = model.eval()

# Moving to training device
model = model.to(DEVICE)

## Dog Loading

In [13]:
dogInfo = pd.read_csv('attributes.csv', index_col=0)

In [14]:
dogInfo = dogInfo.drop_duplicates()

In [15]:
dogInfo = dogInfo[(dogInfo['path'] == 'dog_3') | (dogInfo['path'] == 'dog_8')]

In [16]:
# dogInfo = dogInfo[(dogInfo['path'] == 'dog_8')]

In [17]:
dogInfo

,name,breed,age,sex,size,path,numImages,downloadedImages,link,cleanImages
2,millie,husky & german shepherd dog mix,young,female,medium,dog_3,5,5,https://www.petfinder.com/dog/millie-54650265/...,2
5,sol,labrador retriever mix,adult,male,large,dog_8,4,4,https://www.petfinder.com/dog/sol-52421267/bc/...,2


## Dog Cleaning

In [11]:
numCleanedImagesAll = []

# Iterating through each dog
for index, dog in dogInfo.iterrows():
    
    # Defining list to hold path to i
    imagePaths = []
    
    # Appending the path to each image
    for i in range(0, dog['downloadedImages']):
        imagePaths.append(f"dogs/{dog['path']}/img_{i}.jpg")
        
    # Loading and transforming images
    images = to_tensor(imagePaths)
    
    # Moving images to DEVICE
    images = list(image.to(DEVICE) for image in images)
    
    # Running image through model
    outputs = model(images, '')
    
    # Moving images to CPU
    images = list(image.to(CPU_DEVICE) for image in images)
    
    # Moving images back to CPU
    outputs = [{k: v.to(CPU_DEVICE) for k, v in t.items()} for t in outputs]
    
    # Executing non max surpression on the model output
    outputs = nms(outputs)
    
    # Defining counter of the number of cleaned images
    numCleanedImages = 0
    
    # Iterating through the results and removing the bad pictures
    for i in range(0, dog['downloadedImages']):
        # If the image is not clean (i.e. no dog or more than 1 dog, we delete it)
        if len(outputs[i]['labels']) != 1:
            os.remove(f"dogs/{dog['path']}/img_{i}.jpg")
        
        # If the image is clean, we just rename it to keep it inline with numCleanedImagesAll
        else:
            os.rename(f"dogs/{dog['path']}/img_{i}.jpg", f"dogs/{dog['path']}/img_{numCleanedImages}.jpg")
            numCleanedImages += 1
            
    # Appending the cleaned image counts to the list
    numCleanedImagesAll.append(numCleanedImages)
    
# Appending the cleaned image counts to the df
dogInfo['cleanImages'] = numCleanedImagesAll

/home/aidan/Programs/miniconda3/envs/dogapp/lib/python3.9/site-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811803361/work/aten/src/ATen/native/TensorShape.cpp:2157.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


In [12]:
dogInfo.to_csv('attributes.csv')

In [ ]:
# YOU DON"T NEED THIS PART ITS JUST FOR FUN

from plot_tensor import plot_tensor
import numpy as np

# Plotting image just for fun (you don't need this)
for i in range(0, len(images)):
    plot_tensor(images[i],np.array(outputs[i]['boxes']))